# MATCHING MARKETS

Some theorems (courtesy of and using definitions from Harvard CS134):

* <strong>Thm0 (Existence of a Perfect Matching)</strong> :
    * A bipartite network $G$
        * (a network whose nodes are separable into two subsets $U$ and $V$
        * such that the neighborhood $N(u)$ of any node $u \in U$
        * does not include any other node $u' \in U$,
        * and similarly for $V$)
    * with $|U| = |V|$
    * has a perfect matching
    * iff it does not contain a set of constricted nodes
        * (a subset of nodes $U_C \subseteq U$
        * such that the neighborhood $N(U_C) = \bigcup_{u \in U_C} N(u)$ of $U_R$
        * is smaller than the number of nodes $|U_C|$---
        * or similarly for $V$).


* <strong>Thm1a (Existence of a Set of Socially-Optimal Market-Clearing Prices)</strong> :
    * For any set of buyer valuations (nonnegative real numbers) for each seller,
    * there exists a set of market-clearing seller prices (nonnegative real numbers)
    * that produces a socially-optimal outcome, i.e.
        * the induced perfect matching in the resulting preferred-seller network
            * (a bipartite subnetwork of the main network $G$
            * such that each buyer is linked to at most one seller --
            * either a single seller who maximizes the payoff to that buyer, with ties broken arbitrarily
            * or, if no seller can provide a nonnegative payoff, then no seller at all)
        * maximizes the sum of payoffs to all buyers and sellers.


* <strong>Thm1b (Social Optimality of Any Set of Market-Clearing Prices)</strong> :
    * For any set of market-clearing prices,
    * any perfect matching in the resulting preferred-seller network
    * is socially-optimal.

In [2]:
# data structures
import pandas as pd

In [20]:
SellerPrices = pd.Series
# each row is a buyer,
# each column is that buyer's valuation for corresponding seller
BuyerVals = pd.DataFrame

class BuyerSellerNetwork:
    def __init__(self, seller_prices: SellerPrices, buyer_vals: BuyerVals):
        self.seller_prices = SellerPrices(seller_prices, dtype=float)
        self.buyer_vals = BuyerVals(buyer_vals, dtype=float)
        self._validate()
    
    def _validate(self) -> bool:
        # seller prices can be NaN
        assert not self.buyer_vals.isnull().any().any(), \
            f"Buyer valuations\n{self.buyer_vals}\ncontain NaN's!"
        # each buyer assigns exactly one valuation to each seller
        assert len(buyer_vals.columns) == len(seller_prices.index), \
            f"There are {len(self.seller_prices.index)} sellers, but " + \
            f"{len(self.buyer_vals.columns)} buyer valuations!"
        # the number of buyers is same as number of sellers
        assert len(buyer_vals.index) == len(seller_prices.index), \
            f"There are {len(self.seller_prices.index)} sellers, but " + \
            f"{len(self.buyer_vals.index)} buyers!"
        return True
    
    def set_sellers_price(self, seller: int, price: float) -> bool:
        self.seller_prices.loc[seller] = price
        return True
    
    def __repr__(self):
        return \
            f"Seller prices:\n{self.seller_prices},\n\n" + \
            f"Buyer valuations:\n{self.buyer_vals}"


seller_prices = SellerPrices([None, None, None], dtype=float)
buyer_vals = BuyerVals([
    [0, 1, 2],
    [2, 1, 0],
    [1, 2, 1]
])
g = BuyerSellerNetwork(seller_prices=seller_prices, buyer_vals=buyer_vals)
g

Seller prices:
0   NaN
1   NaN
2   NaN
dtype: float64,

Buyer valuations:
     0    1    2
0  0.0  1.0  2.0
1  2.0  1.0  0.0
2  1.0  2.0  1.0

In [21]:
class PricedBuyerSellerNetwork(BuyerSellerNetwork):
    def _validate(self) -> bool:
        # seller prices can't be None here
        assert not self.seller_prices.isnull().any(), \
            f"Seller prices\n{self.seller_prices}\ncontain NaN's!"
        super()._validate()
        return True


# this will fail -- we have NaN prices!
pg = PricedBuyerSellerNetwork(seller_prices=seller_prices, buyer_vals=buyer_vals)
pg

AssertionError: Seller prices
0   NaN
1   NaN
2   NaN
dtype: float64
contain NaN's!

# Auction algorithm